# Clade Labelling

**This notebook is functional and actually very complete, but further development has been moved over to `aus-h3n2`, because I want to further aggressively simplify clade-labelling to just a few function calls.** There's currently still too much post-processing before passing to baltic. In the meantime, this is kept here for pedagogic purposes.

Development workbook for clade-labelling functions. 

### Input
 - a tree file, with some reference names embedded in there.
 - list of reference names

### Output
 - A dataframe with a list of tipnames, and their clade labels. 

It's possible for a sequence to not have a clade label if that tipname is too far away from even the nearest reference to be meaningfully determined to "belong" to that reference. 

### Notes

Is it possible to write a `def` which outputs a standard plot of a tree and bar chart of distance from nearest reference? Tried a couple of times, but encountered input-hell.

Also note that `Bio.Phylo` will include quotation marks in the tipnames; Figtree doesn't do this.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.collections import PatchCollection, LineCollection
import matplotlib.path as mpath
import matplotlib.lines as mlines
import seaborn as sns

import numpy as np
import pandas as pd
import os
os.chdir("/users/dten0001/Google Drive/baltic3/examples")
import sys
from functools import reduce
import subprocess
import time

# Set random state
my_randomstate = np.random.RandomState(12345)

import xio
import my_utils as xu
import biophylo_utils as bpu
import baltic3 as bt
import baltic3_utils as btu

from Bio import Phylo


In [ ]:
tre = Phylo.read("H3_test_tree.nex", 'nexus')
#tree.root_at_midpoint()
#tree.ladderize()

names_ls = [x.name for x in tre.get_terminals()]
ref_names_ls = []
for nm in names_ls:
    if nm.split("|")[0] == "'A/H3N2":
        ref_names_ls.append(nm)
        
ref_names_ls = ["'A/H3N2|A/Singapore/INFIMH-16-0019/2016|Singapore|2016-06-14'",
                "'A/H3N2|A/HongKong/4801/2014|HongKong|2014'",
                "'A/H3N2|A/SouthAustralia/55/2014|Australia|2014-06-29'",
                "'A/H3N2|A/SouthAustralia/30/2012|Australia|2012-05-10'",
                "'A/H3N2|A/Victoria/361/2011|Australia|2011-10-24'"]

df = bpu.get_clade_labels(tre, ref_names_ls)

# Fix up df a bit more to make referencing easier
df["clade_label_iso_name"] = df.apply(lambda row: str(row["clade_label"]).split("|")[1], axis=1)
for ln in list(set(df["clade_label_iso_name"])):
    print(ln)

# show statistics and plots of distance to reference
for ref_iso_nm in list(set(df["clade_label_iso_name"])):
    print(ref_iso_nm)
    d_temp = df.loc[df["clade_label_iso_name"]==ref_iso_nm]
    min_dist_arr = np.array(d_temp["min_dist"])

    for q in np.arange(80, 100, 5):
        print("%s-th percentile = %.4f" % (q, np.percentile(min_dist_arr, q)))

    print("")

No. of reference tip names = 5
No. of non-reference tip names = 595


In [ ]:
# Write out
#df.to_csv("H3_clade_labels.csv", index=False)

# Plot!

In [ ]:
# show baltic tree and bar chart
bt_tre = btu.austechia_read_tree("H3_test_tree.nex", date_delim="|")

branchWidth=0.5
track_width = 0.0065
track_x0 = bt_tre.treeHeight*1.1 # abs value of position of left side of left-most track
s=10 # tip shape size

track_x = bt_tre.treeHeight*1.07

# clade label colour dictionary
label_cdict = {"A/Singapore/INFIMH-16-0019/2016":"red", 
               'A/Victoria/361/2011':"blue",
              'A/Stockholm/6/2014':"green", 
              'A/Switzerland/9715293/2013':"purple", # Not used
               'A/SouthAustralia/30/2012':"orange", 
              'A/HongKong/4801/2014':"cyan", 
              'A/SouthAustralia/55/2014':"grey"}

ref_names_ls2 = [x.replace("'", "") for x in ref_names_ls]

In [ ]:
fig,ax = plt.subplots(figsize=(11,14),facecolor='w')

lines_ls = []

for k in bt_tre.Objects:
    c = 'k'
    x=k.height # raw (x, y) coords
    y=k.y

    xp = k.parent.height
    if x is None: # matplotlib won't plot Nones, like root
        x = 0
    if xp==None:
        xp = x

    if isinstance(k,bt.leaf) or k.branchType=='leaf':        
        # Draw horizontal bars (not including references)
        if k.name.split("|")[0] != "A/H3N2":
            bar_width = df.loc[df["tip_name"]=="'"+k.name+"'"]["min_dist"].values[0]
            clade_label = df.loc[df["tip_name"]=="'"+k.name+"'"]["clade_label_iso_name"].values[0].replace("'", "")
            ax.plot([track_x, track_x+bar_width], [y, y], color=label_cdict[clade_label], lw=branchWidth*2)
        # Demarcate reference locations
        # Not all reference need be printed out; only user-selected ones
        else:
            if k.name in ref_names_ls2:
                ax.scatter(x,y,s=40,
                           facecolor=label_cdict[k.name.split("|")[1]],
                           edgecolor='none',
                           zorder=11, 
                           alpha=0.4)
                ax.text(x, y, 
                        k.name.split("|")[1], 
                        color="k",
                        size=14)

    elif isinstance(k,bt.node) or k.branchType=='node':
        line = np.array([[x, k.children[0].y], [x, k.children[-1].y]])
        lines_ls.append(line)

    line = np.array([[xp, y], [x, y]])
    lines_ls.append(line)
        
line_collection = LineCollection(lines_ls, lw=branchWidth,color='k', zorder=9)
ax.add_collection(line_collection)


# Switch off the axis
plt.axis("off")
plt.savefig("H3_test.pdf")
plt.show()

## Seaborn Heatmap

In [ ]:
df = df.sort_values(by="clade_label_iso_name")
d_temp = df[['tip_name',
          "dist_to_'A/H3N2|A/Singapore/INFIMH-16-0019/2016|Singapore|2016-06-14'",
          "dist_to_'A/H3N2|A/HongKong/4801/2014|HongKong|2014'",
          "dist_to_'A/H3N2|A/SouthAustralia/55/2014|Australia|2014-06-29'",
          "dist_to_'A/H3N2|A/SouthAustralia/30/2012|Australia|2012-05-10'",
          "dist_to_'A/H3N2|A/Victoria/361/2011|Australia|2011-10-24'"]]

new_colnames = ["A/SG/2016", "A/HK/4801/2014", "A/SthAus/55/2014", "A/SthAus/30/2012", "A/Vic/361/2011"]

d_temp.columns=["tip_name"]+new_colnames

plt.figure(figsize=(5, 18))
sns.heatmap(d_temp[new_colnames])
plt.show()